# Les 6: Gradient Descent

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 6.0 Recap en Motivatie

In de vorige les hebben we geleerd hoe we afgeleiden berekenen: hoe een functie verandert als we de input veranderen. We kunnen nu berekenen hoe de output van een neuron verandert als we de weights aanpassen.

Maar weten hoeveel iets verandert is niet hetzelfde als weten wat de beste waarde is. Het doel van een neuraal netwerk is om de "fout" te minimaliseren - de discrepantie tussen wat het netwerk voorspelt en wat de werkelijke waarde is. Dit is een optimalisatieprobleem.

Gradient descent is het standaard algoritme om dit te doen. Het idee is verrassend simpel: als de gradiënt je vertelt welke kant omhoog is, ga dan de andere kant op. Stap voor stap dalen we af naar het minimum.

## 6.1 Leerdoelen

Na deze les begrijp je het optimalisatieprobleem van neurale netwerken. Je kunt loss functions definiëren en berekenen. Je begrijpt gradient descent intuïtief en wiskundig. Je kent het effect van de learning rate. Je kent de varianten: batch, stochastic en mini-batch gradient descent. Je kunt gradient descent implementeren voor eenvoudige problemen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

## 6.2 Loss Functions

### Waarom hebben we een loss functie nodig?

Een loss functie (ook wel cost functie of objective functie genoemd) is een maat voor hoe "slecht" het model presteert. Het is een enkel getal dat aangeeft hoever de voorspellingen van het model afwijken van de werkelijke waarden.

Het doel van training is deze loss te minimaliseren. De loss is een functie van alle parameters (weights en biases) van het netwerk: L(w₁, w₂, ..., wₙ).

### Mean Squared Error (MSE)

Voor regressieproblemen (waar we een continue waarde voorspellen) is Mean Squared Error de meest gebruikte loss:

MSE = (1/n) Σ (voorspelling - werkelijk)²

We nemen het kwadraat zodat positieve en negatieve fouten niet tegen elkaar wegvallen, en zodat grote fouten zwaarder wegen.

In [ ]:
# Mean Squared Error
def mse_loss(y_pred, y_true):
    """Bereken Mean Squared Error."""
    return np.mean((y_pred - y_true) ** 2)

# Voorbeeld
y_true = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
y_pred_good = np.array([1.1, 2.0, 2.9, 4.1, 5.0])  # Goede voorspelling
y_pred_bad = np.array([2.0, 3.5, 1.0, 5.0, 3.0])   # Slechte voorspelling

print("Werkelijke waarden:", y_true)
print()
print("Goede voorspelling: ", y_pred_good)
print(f"MSE = {mse_loss(y_pred_good, y_true):.4f}")
print()
print("Slechte voorspelling:", y_pred_bad)
print(f"MSE = {mse_loss(y_pred_bad, y_true):.4f}")
print()
print("Lagere MSE = betere voorspelling!")

### Cross-Entropy Loss

Voor classificatieproblemen gebruiken we vaak Cross-Entropy Loss. Dit meet hoe ver de voorspelde kansverdeling afwijkt van de werkelijke verdeling.

Voor binaire classificatie:
L = -[y·log(p) + (1-y)·log(1-p)]

waarbij y de werkelijke klasse is (0 of 1) en p de voorspelde kans is.

In [ ]:
# Binary Cross-Entropy
def binary_cross_entropy(y_pred, y_true, epsilon=1e-15):
    """Bereken Binary Cross-Entropy Loss."""
    # Clip om log(0) te voorkomen
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Voorbeeld: classificatie met 2 klassen
y_true = np.array([1, 0, 1, 1, 0])  # Werkelijke klassen

# Goede voorspellingen (hoge kans voor klasse 1 waar y=1)
y_pred_good = np.array([0.9, 0.1, 0.8, 0.95, 0.2])

# Slechte voorspellingen
y_pred_bad = np.array([0.4, 0.6, 0.3, 0.5, 0.7])

print("Werkelijke klassen:", y_true)
print()
print("Goede voorspellingen:", y_pred_good)
print(f"Cross-Entropy = {binary_cross_entropy(y_pred_good, y_true):.4f}")
print()
print("Slechte voorspellingen:", y_pred_bad)
print(f"Cross-Entropy = {binary_cross_entropy(y_pred_bad, y_true):.4f}")

## 6.3 Het Optimalisatielandschap

De loss functie kan worden gezien als een "landschap" over de parameterruimte. Elke combinatie van parameters geeft een bepaalde loss, en we zoeken het punt met de laagste loss.

Laten we dit visualiseren voor een simpele functie met twee parameters.

In [ ]:
# Visualiseer een loss landschap
# L(w1, w2) = (w1 - 2)² + (w2 - 3)² + 1
# Minimum op (2, 3) met waarde 1

def loss_landscape(w1, w2):
    return (w1 - 2)**2 + (w2 - 3)**2 + 1

# Maak een grid
w1_range = np.linspace(-2, 6, 100)
w2_range = np.linspace(-1, 7, 100)
W1, W2 = np.meshgrid(w1_range, w2_range)
L = loss_landscape(W1, W2)

# 3D plot
fig = plt.figure(figsize=(14, 5))

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(W1, W2, L, cmap='viridis', alpha=0.8)
ax1.set_xlabel('w₁')
ax1.set_ylabel('w₂')
ax1.set_zlabel('Loss')
ax1.set_title('Loss landschap (3D)')

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contour(W1, W2, L, levels=20, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.plot(2, 3, 'r*', markersize=15, label='Minimum (2, 3)')
ax2.set_xlabel('w₁')
ax2.set_ylabel('w₂')
ax2.set_title('Loss landschap (contour)')
ax2.legend()

plt.tight_layout()
plt.show()

print("Het doel is om van een willekeurig startpunt naar het minimum te komen.")

In [ ]:
# Een complexer landschap met lokale minima
def complex_landscape(w1, w2):
    return np.sin(w1) * np.cos(w2) + 0.1 * (w1**2 + w2**2)

w1_range = np.linspace(-4, 4, 100)
w2_range = np.linspace(-4, 4, 100)
W1, W2 = np.meshgrid(w1_range, w2_range)
L_complex = complex_landscape(W1, W2)

fig = plt.figure(figsize=(14, 5))

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(W1, W2, L_complex, cmap='coolwarm', alpha=0.8)
ax1.set_xlabel('w₁')
ax1.set_ylabel('w₂')
ax1.set_zlabel('Loss')
ax1.set_title('Complex landschap met meerdere minima')

ax2 = fig.add_subplot(122)
contour = ax2.contour(W1, W2, L_complex, levels=30, cmap='coolwarm')
ax2.set_xlabel('w₁')
ax2.set_ylabel('w₂')
ax2.set_title('Contour - let op de meerdere "dalen"')

plt.tight_layout()
plt.show()

print("In de praktijk kan het landschap zeer complex zijn.")
print("We kunnen vastlopen in lokale minima.")

## 6.4 Gradient Descent Algoritme

### Het idee

De gradiënt van de loss functie wijst in de richting van de steilste stijging. Als we de loss willen minimaliseren, moeten we dus de tegengestelde richting op: de negatieve gradiënt.

### De update regel

w_new = w_old - η · ∇L(w)

waarbij η (eta) de learning rate is, een hyperparameter die bepaalt hoe grote stappen we nemen.

### Het algoritme

1. Initialiseer parameters willekeurig
2. Bereken de gradiënt van de loss
3. Update de parameters: w = w - η · ∇L
4. Herhaal stap 2-3 tot convergentie

In [ ]:
# Gradient descent voor een simpele 1D functie
# f(x) = x² - 4x + 5 = (x-2)² + 1
# Minimum op x = 2

def f(x):
    return x**2 - 4*x + 5

def f_gradient(x):
    return 2*x - 4

# Gradient descent
def gradient_descent_1d(f, f_grad, x_init, learning_rate, n_iterations):
    x = x_init
    history = [x]
    
    for i in range(n_iterations):
        grad = f_grad(x)
        x = x - learning_rate * grad
        history.append(x)
    
    return x, history

# Run
x_init = 6.0
learning_rate = 0.1
n_iter = 20

x_final, history = gradient_descent_1d(f, f_gradient, x_init, learning_rate, n_iter)

print(f"Start: x = {x_init}, f(x) = {f(x_init)}")
print(f"Eind:  x = {x_final:.6f}, f(x) = {f(x_final):.6f}")
print(f"Optimum: x = 2, f(x) = 1")

In [ ]:
# Visualisatie van het pad
x_range = np.linspace(-1, 7, 100)

plt.figure(figsize=(12, 5))

# Links: functie met pad
plt.subplot(121)
plt.plot(x_range, f(x_range), 'b-', linewidth=2, label='f(x) = x² - 4x + 5')
history_arr = np.array(history)
plt.plot(history_arr, f(history_arr), 'ro-', markersize=8, alpha=0.7, label='GD pad')
plt.plot(history_arr[0], f(history_arr[0]), 'g^', markersize=15, label='Start')
plt.plot(2, 1, 'r*', markersize=20, label='Minimum')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Gradient Descent op f(x) = x² - 4x + 5')
plt.legend()
plt.grid(True, alpha=0.3)

# Rechts: convergentie
plt.subplot(122)
plt.plot(range(len(history)), f(np.array(history)), 'b-', linewidth=2)
plt.xlabel('Iteratie')
plt.ylabel('f(x)')
plt.title('Convergentie van de loss')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 2D Gradient Descent
def gradient_descent_2d(loss_fn, grad_fn, w_init, learning_rate, n_iterations):
    w = np.array(w_init, dtype=float)
    history = [w.copy()]
    
    for i in range(n_iterations):
        grad = grad_fn(w[0], w[1])
        w = w - learning_rate * np.array(grad)
        history.append(w.copy())
    
    return w, history

# Loss: L(w1, w2) = (w1-2)² + (w2-3)² + 1
def loss_2d(w1, w2):
    return (w1 - 2)**2 + (w2 - 3)**2 + 1

def grad_2d(w1, w2):
    return [2*(w1 - 2), 2*(w2 - 3)]

# Run
w_init = [-1, 6]
w_final, history_2d = gradient_descent_2d(loss_2d, grad_2d, w_init, 0.1, 30)

print(f"Start: w = {w_init}, Loss = {loss_2d(*w_init):.4f}")
print(f"Eind:  w = [{w_final[0]:.4f}, {w_final[1]:.4f}], Loss = {loss_2d(*w_final):.4f}")

In [ ]:
# Visualisatie 2D GD
w1_range = np.linspace(-2, 6, 100)
w2_range = np.linspace(-1, 8, 100)
W1, W2 = np.meshgrid(w1_range, w2_range)
L = loss_2d(W1, W2)

plt.figure(figsize=(10, 8))
plt.contour(W1, W2, L, levels=30, cmap='viridis')
plt.colorbar(label='Loss')

# Plot het pad
history_arr = np.array(history_2d)
plt.plot(history_arr[:, 0], history_arr[:, 1], 'ro-', markersize=6, linewidth=2, label='GD pad')
plt.plot(history_arr[0, 0], history_arr[0, 1], 'g^', markersize=15, label='Start')
plt.plot(2, 3, 'r*', markersize=20, label='Minimum')

plt.xlabel('w₁', fontsize=12)
plt.ylabel('w₂', fontsize=12)
plt.title('Gradient Descent in 2D', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## 6.5 De Learning Rate

De learning rate η is een cruciale hyperparameter. Het bepaalt hoe grote stappen we nemen in de richting van de negatieve gradiënt.

- **Te groot**: We kunnen over het minimum heen schieten (overshoot), oscilleren, of zelfs divergeren.
- **Te klein**: Convergentie is zeer traag, en we kunnen vastlopen in lokale minima.
- **Just right**: Snelle, stabiele convergentie.

In [ ]:
# Effect van verschillende learning rates
learning_rates = [0.01, 0.1, 0.5, 0.9]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

x_range = np.linspace(-1, 7, 100)

for ax, lr in zip(axes.flatten(), learning_rates):
    x_final, history = gradient_descent_1d(f, f_gradient, 6.0, lr, 20)
    history_arr = np.array(history)
    
    ax.plot(x_range, f(x_range), 'b-', linewidth=2)
    ax.plot(history_arr, f(history_arr), 'ro-', markersize=6, alpha=0.7)
    ax.plot(2, 1, 'r*', markersize=15)
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.set_title(f'Learning rate = {lr}')
    ax.set_ylim(-1, 30)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observaties:")
print("- lr=0.01: Te langzaam, convergeert nog niet na 20 stappen")
print("- lr=0.1:  Goede balans, convergeert netjes")
print("- lr=0.5:  Snel maar begint te oscilleren")
print("- lr=0.9:  Oscilleert wild, convergeert nauwelijks")

In [ ]:
# Convergentie curves vergelijken
plt.figure(figsize=(10, 6))

for lr in [0.01, 0.05, 0.1, 0.3, 0.5]:
    _, history = gradient_descent_1d(f, f_gradient, 6.0, lr, 50)
    plt.plot(f(np.array(history)), label=f'lr={lr}')

plt.xlabel('Iteratie', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Effect van learning rate op convergentie', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

## 6.6 Varianten van Gradient Descent

In de praktijk hebben we niet één data punt maar een hele dataset. Er zijn drie hoofdvarianten voor hoe we de gradiënt berekenen:

### Batch Gradient Descent
Bereken de gradiënt over de **hele dataset** voor elke update. 
- ✓ Nauwkeurige gradiënt
- ✗ Traag voor grote datasets
- ✗ Kan vastlopen in lokale minima

### Stochastic Gradient Descent (SGD)
Bereken de gradiënt op **één willekeurig sample** per update.
- ✓ Snel per iteratie
- ✓ Ruis helpt ontsnappen aan lokale minima
- ✗ Zeer ruisige updates, kan oscilleren

### Mini-batch Gradient Descent
Bereken de gradiënt op een **kleine batch** (bijv. 32 samples) per update.
- ✓ Beste van beide werelden
- ✓ Efficiënt (GPU parallelisatie)
- Dit is de standaard in deep learning!

In [ ]:
# Simulatie van de drie varianten voor lineaire regressie

# Genereer data
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 1)
y_true = 3 * X[:, 0] + 2 + np.random.randn(n_samples) * 0.5

# Model: y = w*x + b
def predict(X, w, b):
    return X[:, 0] * w + b

def mse(X, y, w, b):
    return np.mean((predict(X, w, b) - y) ** 2)

def gradient(X, y, w, b):
    n = len(y)
    y_pred = predict(X, w, b)
    dw = (2/n) * np.sum((y_pred - y) * X[:, 0])
    db = (2/n) * np.sum(y_pred - y)
    return dw, db

# Batch GD
def batch_gd(X, y, w_init, b_init, lr, n_epochs):
    w, b = w_init, b_init
    history = []
    for epoch in range(n_epochs):
        dw, db = gradient(X, y, w, b)
        w -= lr * dw
        b -= lr * db
        history.append(mse(X, y, w, b))
    return w, b, history

# SGD
def sgd(X, y, w_init, b_init, lr, n_epochs):
    w, b = w_init, b_init
    history = []
    for epoch in range(n_epochs):
        # Shuffle data
        indices = np.random.permutation(len(y))
        for i in indices:
            Xi = X[i:i+1]
            yi = y[i:i+1]
            dw, db = gradient(Xi, yi, w, b)
            w -= lr * dw
            b -= lr * db
        history.append(mse(X, y, w, b))
    return w, b, history

# Mini-batch GD
def minibatch_gd(X, y, w_init, b_init, lr, n_epochs, batch_size=16):
    w, b = w_init, b_init
    history = []
    for epoch in range(n_epochs):
        indices = np.random.permutation(len(y))
        for start in range(0, len(y), batch_size):
            idx = indices[start:start+batch_size]
            Xi = X[idx]
            yi = y[idx]
            dw, db = gradient(Xi, yi, w, b)
            w -= lr * dw
            b -= lr * db
        history.append(mse(X, y, w, b))
    return w, b, history

In [ ]:
# Vergelijk de drie methodes
w_init, b_init = 0.0, 0.0
n_epochs = 50

w_batch, b_batch, hist_batch = batch_gd(X, y_true, w_init, b_init, 0.1, n_epochs)
w_sgd, b_sgd, hist_sgd = sgd(X, y_true, w_init, b_init, 0.01, n_epochs)  # Kleinere lr voor SGD
w_mini, b_mini, hist_mini = minibatch_gd(X, y_true, w_init, b_init, 0.1, n_epochs, batch_size=16)

print("Resultaten na 50 epochs:")
print(f"Werkelijke parameters: w=3, b=2")
print()
print(f"Batch GD:     w={w_batch:.4f}, b={b_batch:.4f}, MSE={hist_batch[-1]:.4f}")
print(f"SGD:          w={w_sgd:.4f}, b={b_sgd:.4f}, MSE={hist_sgd[-1]:.4f}")
print(f"Mini-batch:   w={w_mini:.4f}, b={b_mini:.4f}, MSE={hist_mini[-1]:.4f}")

In [ ]:
# Visualisatie
plt.figure(figsize=(10, 6))
plt.plot(hist_batch, 'b-', linewidth=2, label='Batch GD')
plt.plot(hist_sgd, 'r-', alpha=0.7, label='SGD')
plt.plot(hist_mini, 'g-', linewidth=2, label='Mini-batch GD')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MSE Loss', fontsize=12)
plt.title('Vergelijking van GD varianten', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("SGD is ruisiger maar convergeert naar dezelfde oplossing.")
print("Mini-batch is een goede balans.")

## 6.7 Toepassing: Linear Regression met Gradient Descent

Laten we een volledige lineaire regressie implementeren met gradient descent en visualiseren hoe het model leert.

In [ ]:
# Genereer data
np.random.seed(42)
n_samples = 50

X_train = np.random.uniform(0, 10, n_samples)
y_train = 2.5 * X_train + 3 + np.random.randn(n_samples) * 2

plt.figure(figsize=(10, 6))
plt.scatter(X_train, y_train, alpha=0.7)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Training data voor lineaire regressie')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Lineaire regressie met gradient descent
class LinearRegressionGD:
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
        self.w = 0.0
        self.b = 0.0
        self.history = []
    
    def predict(self, X):
        return self.w * X + self.b
    
    def loss(self, X, y):
        return np.mean((self.predict(X) - y) ** 2)
    
    def fit(self, X, y, n_epochs=100, verbose=True):
        n = len(y)
        
        for epoch in range(n_epochs):
            # Forward
            y_pred = self.predict(X)
            
            # Gradiënten
            dw = (2/n) * np.sum((y_pred - y) * X)
            db = (2/n) * np.sum(y_pred - y)
            
            # Update
            self.w -= self.lr * dw
            self.b -= self.lr * db
            
            # Log
            current_loss = self.loss(X, y)
            self.history.append({
                'epoch': epoch,
                'loss': current_loss,
                'w': self.w,
                'b': self.b
            })
            
            if verbose and epoch % 20 == 0:
                print(f"Epoch {epoch:3d}: Loss = {current_loss:.4f}, w = {self.w:.4f}, b = {self.b:.4f}")
        
        return self

# Train
model = LinearRegressionGD(learning_rate=0.01)
model.fit(X_train, y_train, n_epochs=100)

print()
print(f"Geleerde parameters: w = {model.w:.4f}, b = {model.b:.4f}")
print(f"Werkelijke relatie:  y = 2.5x + 3")

In [ ]:
# Visualiseer de fit en het leerproces
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Data met fit
axes[0].scatter(X_train, y_train, alpha=0.7, label='Data')
x_line = np.linspace(0, 10, 100)
axes[0].plot(x_line, model.predict(x_line), 'r-', linewidth=2, label=f'Fit: y = {model.w:.2f}x + {model.b:.2f}')
axes[0].plot(x_line, 2.5*x_line + 3, 'g--', linewidth=2, alpha=0.5, label='Werkelijk: y = 2.5x + 3')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')
axes[0].set_title('Lineaire Regressie Fit')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss curve
losses = [h['loss'] for h in model.history]
axes[1].plot(losses, 'b-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MSE Loss')
axes[1].set_title('Training Loss')
axes[1].grid(True, alpha=0.3)

# Parameter evolutie
ws = [h['w'] for h in model.history]
bs = [h['b'] for h in model.history]
axes[2].plot(ws, label='w')
axes[2].plot(bs, label='b')
axes[2].axhline(y=2.5, color='blue', linestyle='--', alpha=0.5)
axes[2].axhline(y=3, color='orange', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Parameter waarde')
axes[2].set_title('Parameter Evolutie')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6.8 Samenvatting en Vooruitblik

### Kernconcepten

Een loss functie meet hoe slecht het model presteert. MSE voor regressie, Cross-Entropy voor classificatie. Het doel is de loss te minimaliseren.

Gradient descent is het standaard optimalisatie-algoritme: update parameters in de richting van de negatieve gradiënt. De update regel is w = w - η·∇L.

De learning rate η bepaalt de stapgrootte. Te groot = instabiliteit, te klein = trage convergentie.

Mini-batch gradient descent is de standaard in deep learning: een goede balans tussen nauwkeurigheid en snelheid.

### Wat ontbreekt nog?

We hebben gradient descent toegepast op eenvoudige problemen waar we de gradiënt makkelijk konden berekenen. Maar in een diep neuraal netwerk met miljoenen parameters, hoe berekenen we dan de gradiënt van de loss naar elke parameter?

### Volgende les

In les 7 leren we backpropagation: het algoritme dat de kettingregel efficiënt toepast om gradiënten door het hele netwerk te berekenen. We zullen een volledig neuraal netwerk trainen op MNIST!

### Checklist

Controleer of je het volgende begrijpt:

1. Wat is een loss functie en waarom hebben we deze nodig?

2. Wat is de update regel van gradient descent?

3. Wat gebeurt er als de learning rate te groot of te klein is?

4. Wat is het verschil tussen batch, stochastic en mini-batch GD?

5. Waarom is mini-batch GD de standaard?

Als je deze vragen kunt beantwoorden, ben je klaar voor les 7!

---

**Mathematical Foundations** | Les 6 van 12 | IT & Artificial Intelligence

---